# A full business solution

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import google.generativeai as genai

C:\Users\Welcome\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')  # Make sure to set this in your .env file

if api_key and len(api_key) > 10:
    print("Gemini API key looks good so far")
    genai.configure(api_key=api_key)
else:
    print("There might be a problem with your Gemini API key? Please check your .env file!")
MODEL = 'gemma-3-27b-it'

Gemini API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
import json
import re
def get_links(url):
    website = Website(url)
    # Combine system and user prompts into one string
    prompt = link_system_prompt + "\n" + get_links_user_prompt(website)
    response = genai.GenerativeModel(MODEL).generate_content(prompt)
    result = response.text.strip()
    # Remove markdown code block if present
    if result.startswith("```"):
        result = re.sub(r"^```[a-zA-Z]*\n?", "", result)
        result = re.sub(r"\n?```$", "", result)
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/google/gemma-3n-E4B-it',
 '/tencent/Hunyuan-A13B-Instruct',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/kyutai/tts-1.6b-en_fr',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/AIDC-AI/Ovis-U1-3B',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/facebook/seamless-interaction',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets/black-forest-labs/kontext-bench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wri

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'docs page', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn', 'url': 'https://huggingface.co/learn'},
  {'type': 'github', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'endpoints', 'url': 'https://endpoints.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn', 'url': 'https://huggingface.co/learn'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'endpoints', 'url': 'https://endpoints.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community 

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [24]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:7_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn', 'url': 'https://huggingface.co/learn'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'endpoints', 'url': 'https://endpoints.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n9 days ago\n•\n163k\n•\n1.35k\ngoogle/gemma-3n-E4B-it\nUpdated\n3 days ago\n•\n212k\n•\n470\ntencent/Hunyuan-A13B-Instruct\nUpdated\n4 days ago\n•\n20.8k\n•\n719\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n3 days ago\n•\n8.58k\n•\n220\nkyutai/tts-1.6b-en_fr\nUpdated\n3 days ago\n•\n9.29k\n•\n186\nBrowse 1M+ models\nSpaces\nRunning\n9.32k\n9.32k\nDee

In [25]:
def create_brochure(company_name, url):
    import re
    prompt = system_prompt + "\n" + get_brochure_user_prompt(company_name, url)
    model = genai.GenerativeModel(MODEL)
    response = model.generate_content(prompt)
    result = response.text.strip()
    # Remove markdown code block if present
    if result.startswith("```"):
        result = re.sub(r"^```[a-zA-Z]*\n?", "", result)
        result = re.sub(r"\n?```$", "", result)
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn', 'url': 'https://huggingface.co/learn'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'endpoints', 'url': 'https://endpoints.huggingface.co'}]}


## Hugging Face: Democratizing AI - A Brochure

**(Front Panel - Image: A diverse group of people collaborating around a computer screen displaying AI visualizations)**

**Hugging Face – The AI community building the future.**

**What is Hugging Face?**

Hugging Face is the leading open-source and open-science platform for the machine learning community. We provide the tools and resources to build, share, and collaborate on the next generation of AI applications.  With over 1 million models and 400k+ applications available, we're empowering developers, researchers, and businesses to accelerate their AI journey.

---

**(Inside Left Panel - Heading: Powering Innovation)**

**For Customers:**

* **Explore a Universe of AI:** Access over 1 million pre-trained models, 250k+ datasets, and 400k+ Spaces (AI applications) covering text, image, audio, video, and 3D.
* **Accelerate Development:**  Leverage our open-source tools and libraries like Transformers, Diffusers, and Datasets to rapidly prototype and deploy AI solutions.
* **Scalable Solutions:**  From individual projects to enterprise-grade deployments, Hugging Face offers Compute solutions with optimized Inference Endpoints and Spaces applications.
* **Trusted by Leading Organizations:**  Over 50,000 organizations, including AI at Meta, Google, Amazon, and Microsoft, rely on Hugging Face.

**For Investors:**

* **Rapid Growth:** Hugging Face is at the forefront of the booming AI market, experiencing significant growth in users, models, and enterprise adoption.
* **Strong Community:**  A vibrant and engaged community drives innovation and contributes to the platform's continuous improvement.
* **Open-Source Leadership:**  Our commitment to open-source fosters collaboration and accelerates the development of cutting-edge AI technologies.
* **Diverse Revenue Streams:**  Hugging Face generates revenue through Compute, Enterprise solutions, and potential future offerings.

---

**(Inside Right Panel - Heading: Join the Movement)**

**Our Culture:**

At Hugging Face, we believe in **openness, collaboration, and inclusivity**. We are a remote-first company with a global team passionate about democratizing AI. We value:

* **Community First:** We prioritize the needs of our community and actively encourage contributions.
* **Impact Driven:** We focus on building tools that have a real-world impact.
* **Continuous Learning:** We embrace experimentation and are committed to staying at the forefront of AI research.
* **Transparency & Trust:** We operate with honesty and integrity in all our interactions.

**Careers at Hugging Face:**

We're always looking for talented individuals to join our team!  We offer opportunities in:

* **Engineering:**  Building and maintaining our platform and tools.
* **Research:**  Pushing the boundaries of AI technology.
* **Community:**  Supporting and growing our vibrant community.
* **Sales & Marketing:**  Bringing Hugging Face to the world.

**Find open positions and learn more at: [https://huggingface.co/jobs](https://huggingface.co/jobs)**

---

**(Back Panel - Contact Information & Social Media)**

**Hugging Face**
[https://huggingface.co/](https://huggingface.co/)

**Connect with us:**

* **GitHub:** [https://github.com/huggingface](https://github.com/huggingface)
* **Twitter:** [https://twitter.com/huggingface](https://twitter.com/huggingface)
* **LinkedIn:** [https://www.linkedin.com/company/hugging-face/](https://www.linkedin.com/company/hugging-face/)
* **Discord:** [https://discord.gg/huggingface](https://discord.gg/huggingface)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
import time
import re
from IPython.display import Markdown, display, update_display

def stream_brochure(company_name, url):
    prompt = system_prompt + "\n" + get_brochure_user_prompt(company_name, url)
    model = genai.GenerativeModel(MODEL)
    response = model.generate_content(prompt)
    result = response.text.strip()
    # Remove markdown code block if present
    if result.startswith("```"):
        result = re.sub(r"^```[a-zA-Z]*\n?", "", result)
        result = re.sub(r"\n?```$", "", result)
    # Simulate streaming by displaying the result chunk by chunk
    display_handle = display(Markdown(""), display_id=True)
    streamed = ""
    for chunk in result.split('\n'):
        streamed += chunk + '\n'
        update_display(Markdown(streamed), display_id=display_handle.display_id)
        time.sleep(0.05)  # Adjust speed as desired

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn', 'url': 'https://huggingface.co/learn'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'endpoints', 'url': 'https://endpoints.huggingface.co'}]}


## Hugging Face: Democratizing AI - A Brochure

**(Front Panel - Image: A diverse group of people collaborating around a computer screen displaying AI visualizations)**

**Hugging Face – The AI Community Building the Future.**

**What is Hugging Face?**

Hugging Face is the leading open-source and open-science platform for the machine learning community. We provide the tools and resources to build, share, and collaborate on the next generation of AI applications.  With over 1 million models and 400,000 applications available, we're empowering individuals and organizations to accelerate their AI journey.

---

**(Inside Left Panel - Heading: Powering Innovation)**

**For Customers:**

* **Explore a Universe of AI:** Access over 1 million pre-trained models, 250k+ datasets, and 400k+ Spaces (AI applications) covering text, image, audio, video, and 3D.
* **Accelerate Development:** Leverage our open-source tools like Transformers, Diffusers, and Datasets to rapidly prototype and deploy AI solutions.
* **Scalable Solutions:**  Utilize our Compute solutions – Inference Endpoints and Spaces – to deploy and scale your applications with ease.  Starting at just $0.60/hour for GPU access.
* **Trusted by Leading Organizations:**  Companies like Google, Meta, Amazon, and Intel rely on Hugging Face to power their AI initiatives.

**For Investors:**

* **Rapid Growth:**  Hugging Face is at the forefront of the booming AI market, experiencing significant user and organizational adoption.
* **Strong Community:**  A vibrant and engaged community drives innovation and contributes to the platform's continuous improvement.
* **Multiple Revenue Streams:**  Hugging Face generates revenue through Compute, Enterprise solutions, and potential future offerings.
* **Democratizing AI:**  We are making AI accessible to everyone, fostering a more inclusive and innovative future.

---

**(Inside Right Panel - Heading: Join the Movement)**

**Our Culture:**

At Hugging Face, we believe in **openness, collaboration, and inclusivity.** We are a remote-first company with a global team passionate about making AI accessible to all. We value:

* **Community First:** We prioritize the needs of our community and actively encourage contributions.
* **Continuous Learning:** We foster a culture of learning and experimentation.
* **Impactful Work:** We are dedicated to building tools that have a positive impact on the world.
* **Transparency:** We operate with honesty and openness in all our interactions.

**Careers at Hugging Face:**

We're always looking for talented individuals to join our team!  We offer opportunities in:

* **Engineering:** Building and maintaining our platform and tools.
* **Research:**  Pushing the boundaries of AI innovation.
* **Community:**  Supporting and growing our vibrant community.
* **Sales & Marketing:**  Bringing Hugging Face to the world.

**Find open positions and learn more at [https://huggingface.co/jobs](https://huggingface.co/jobs)**

---

**(Back Panel - Contact & Social Media)**

**Connect with Us:**

* **Website:** [https://huggingface.co/](https://huggingface.co/)
* **GitHub:** [https://github.com/huggingface](https://github.com/huggingface)
* **Twitter:** [https://twitter.com/huggingface](https://twitter.com/huggingface)
* **LinkedIn:** [https://www.linkedin.com/company/hugging-face/](https://www.linkedin.com/company/hugging-face/)
* **Discord:** [https://discord.gg/huggingface](https://discord.gg/huggingface)

**Hugging Face:  Building the future of AI, together.**


In [27]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Edward Donner", "https://edwarddonner.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'}, {'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'}, {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'nebula referral', 'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}, {'type': 'project', 'url': 'https://edwarddonner.com/connect-four/'}, {'type': 'project', 'url': 'https://edwarddonner.com/outsmart/'}]}


## Edward Donner & Nebula.io: AI for Human Potential

**(Brochure - Tri-fold Style)**

---

**[Front Panel - Image: A dynamic, abstract representation of interconnected nodes/networks, hinting at AI and human connection]**

**Headline:** **Unlocking Potential with AI**

**Tagline:** *Matching people to purpose, powered by innovation.*

---

**[Inside Left Panel - Focus: About Nebula.io & Edward Donner]**

**About Us**

Nebula.io, co-founded and led by Edward Donner, is revolutionizing the talent acquisition landscape with the power of Artificial Intelligence. We're not just filling jobs; we're connecting people with opportunities where they can thrive and find fulfillment. 

**The Vision:**  Inspired by the Japanese concept of *Ikigai* – finding your reason for being – Nebula.io aims to raise the level of human prosperity by helping individuals discover roles aligned with their passions and skills.  We believe 77% of people deserve more than just a job; they deserve a calling.

**Meet Ed Donner**

Ed is a seasoned entrepreneur and technology leader.  Previously the founder and CEO of untapt, an AI startup acquired in 2021, he brings a wealth of experience in data science, software engineering, and building innovative solutions for the recruitment industry.  He’s passionate about applying AI to real-world problems and is currently focused on transforming how people find their dream jobs.  

*   **CTO & Co-founder, Nebula.io**
*   **Founder & CEO (former), untapt**
*   **Expert in LLMs & Generative AI**

---

**[Inside Center Panel - Focus: What Nebula.io Does & Who We Serve]**

**How We Help**

Nebula.io provides a cutting-edge platform for recruiters, leveraging:

*   **Proprietary LLMs:** Groundbreaking, verticalized Large Language Models specifically designed for talent matching.
*   **Patented Matching Model:**  Superior accuracy and speed in connecting candidates with the right roles – *no keywords required*.
*   **Comprehensive Talent Management:** Tools to source, understand, engage, and manage talent effectively.

**Our Customers**

We partner with forward-thinking recruitment firms to streamline their processes and deliver exceptional results.  Our award-winning platform is already trusted by happy customers and has garnered significant press coverage.

**Key Benefits:**

*   **Increased Efficiency:** Faster and more accurate talent matching.
*   **Improved Candidate Quality:**  Finding candidates who are truly aligned with the role and company culture.
*   **Enhanced Candidate Experience:**  A more personalized and engaging recruitment process.
*   **Data-Driven Insights:**  Actionable data to optimize recruitment strategies.

---

**[Inside Right Panel - Focus: Careers, Connect & Contact]**

**Join the Team!**

We’re a dynamic and innovative team passionate about the future of AI and its potential to impact lives.  We are always looking for talented individuals to join us. 

**We offer:**

*   A challenging and rewarding work environment.
*   The opportunity to work with cutting-edge technology.
*   A culture of learning and growth.
*   The chance to make a real difference.

**Connect With Us:**

*   **LinkedIn:** [LinkedIn Link from Website]
*   **Twitter:** [Twitter Link from Website]
*   **Facebook:** [Facebook Link from Website]
*   **Newsletter:** Subscribe at [Website Address]

**Get in Touch:**

*   **Email:** ed [at] edwarddonner [dot] com
*   **Website:** www.edwarddonner.com



---

**[Back Panel - Image: A subtle background pattern with the Nebula.io logo]**

**Nebula.io:  AI. Talent. Potential.**

**[Small text:]**  © 2025 Edward Donner. All rights reserved.
